In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn import decomposition
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from sklearn import preprocessing
import os


from ipywidgets import widgets
%matplotlib inline

In [5]:
def getFeatures(fileName):
    path = os.getcwd() + '/irc-sphere-sleep-56db93f64661/sphere-sensor-data/' + fileName
    
    if 'gz' in fileName:
        df = pd.read_csv(path, index_col='datetime', compression='gzip')
    else:
        df = pd.read_csv(path, index_col='datetime')

    # arm angle calculation using accelerometer data
    df['angle'] = 180 / np.pi * np.arctan(df['wearable-xl1-z'] / np.sqrt(np.square(df['wearable-xl1-x']) + np.square(df['wearable-xl1-z'])))

    # get only interesting columns
    df = df[['angle', 'wearable-mag-xl1']]

    # remove NaN values
    df = df.dropna()
    # convert index to DatetimeIndex
    df.index = pd.DatetimeIndex(df.index)


    # resample data within interval given in resampleInterval ('1S' = 1 second intervals)
    resampleInterval = '1S'
    minimum = df.resample(resampleInterval).min()
    maximum = df.resample(resampleInterval).max()
    mean = df.resample(resampleInterval).mean()
    std = df.resample(resampleInterval).std()
    summ = df.resample(resampleInterval).sum()
    skew = df.resample(resampleInterval).apply(lambda array : stats.skew(array))
    kurtosis = df.resample(resampleInterval).apply(lambda array: stats.kurtosis(array, fisher=True))

    # rename column labels in DataFrame
    minimum.rename(columns={'angle': 'angle min'}, inplace=True)
    minimum.rename(columns={'wearable-mag-xl1': 'magnitude min'}, inplace=True)
    
    maximum.rename(columns={'angle': 'angle max'}, inplace=True)
    maximum.rename(columns={'wearable-mag-xl1': 'magnitude max'}, inplace=True)
    
    mean.rename(columns={'angle': 'angle mean'}, inplace=True)
    mean.rename(columns={'wearable-mag-xl1': 'magnitude mean'}, inplace=True)
    
    std.rename(columns={'angle': 'angle std'}, inplace=True)
    std.rename(columns={'wearable-mag-xl1': 'magnutude std'}, inplace=True)
    
    summ.rename(columns={'angle': 'angle sum'}, inplace=True)
    summ.rename(columns={'wearable-mag-xl1': 'magnitude sum'}, inplace=True)
    
    skew.rename(columns={'angle': 'angle skew'}, inplace=True)
    skew.rename(columns={'wearable-mag-xl1': 'magnitude skew'}, inplace=True)
    
    kurtosis.rename(columns={'angle': 'angle kurtosis'}, inplace=True)
    kurtosis.rename(columns={'wearable-mag-xl1': 'magnitude kurtosis'}, inplace=True)
    
    # collect DataFrames
    features = [minimum, maximum, mean, std, summ, skew, kurtosis]
    features = pd.concat(features, axis=1)

    
    # for some weird reason new nans appear. 
    # need to drop them to have same shape on what to plot
    features = features.dropna()
    std = std.dropna()
    
    return [features, std, df]

In [7]:
# load features from data over a night
[features, std, df] = getFeatures('data_02.csv.gz')
features

//anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,angle min,magnitude min,angle max,magnitude max,angle mean,magnitude mean,angle std,magnutude std,angle sum,magnitude sum,angle skew,magnitude skew,angle kurtosis,magnitude kurtosis
datetime,,,,,,,,,,,,,,
2016-06-02 22:13:58,22.859250,-0.463627,33.030361,0.177312,28.854759,-0.189835,3.640377,0.226352,230.838075,-1.518684,-0.605520,0.329721,-1.105575,-1.175230
2016-06-02 22:13:59,-9.944248,-0.482232,24.754030,0.486497,9.470956,0.046617,8.498399,0.270547,189.419114,0.932341,-0.544464,-0.137403,-0.029219,-1.028425
2016-06-02 22:14:00,-44.736096,-0.429127,34.369830,0.503024,16.872243,-0.065731,22.571371,0.275611,337.444851,-1.314622,-2.044272,0.435070,3.057950,-0.817843
2016-06-02 22:14:01,-44.951943,-0.290679,-17.704064,0.147172,-39.785939,0.001544,7.132096,0.142252,-795.718771,0.030872,1.794272,-0.796109,2.595877,-0.873519
2016-06-02 22:14:02,-44.975562,-0.193387,43.491519,0.147632,-31.094833,-0.052463,22.117252,0.106876,-621.896663,-1.049255,2.346700,0.243469,4.887210,-1.089365
2016-06-02 22:14:03,-44.367414,-0.201997,30.963757,0.143248,-28.002740,-0.013308,20.930291,0.084392,-560.054792,-0.266158,2.271544,-0.286585,3.862914,0.388280
2016-06-02 22:14:04,-44.927105,-0.200278,39.533105,0.353669,-24.605935,-0.010902,22.414270,0.134303,-492.118697,-0.218044,1.489283,1.448048,1.645529,1.824784
2016-06-02 22:14:05,-28.938286,-0.196097,24.183153,0.234429,5.939529,-0.026311,16.782534,0.112534,118.790585,-0.526226,-0.900505,0.629992,-0.641801,0.180998
2016-06-02 22:14:06,-5.593684,-0.141991,22.710641,0.075773,9.289246,-0.004276,9.443753,0.061718,185.784911,-0.085515,-0.070533,-0.841868,-1.359254,0.138574


In [ ]:
# normalise the values between a 0 1 range

features_array = features.values #return a numpy array
min_max_scalar = preprocessing.MinMaxScaler()
normalise = min_max_scalar.fit_transform(features_array)
normalised_features = pd.DataFrame(normalise)

# rename index and columns
normalised_features.columns = features.columns
normalised_features.index = features.index

normalised_features_backup = normalised_features.copy(deep=True)


In [ ]:
# cluster and label data using K-means into n clusters
# label = KMeans(n_clusters=5, random_state=10).fit_predict(features)
# label

In [ ]:
# make PCA (dimensionality reduction) to 3D and plot

def plots(weight):
    

    normalised_features['angle std'] = normalised_features['angle std'] * weight
    label = KMeans(n_clusters=4, random_state=10).fit_predict(normalised_features)

    # PCA PLOT #
    fig = plt.figure(figsize=(10, 10))
    ax1 = fig.add_subplot(211, projection='3d')
    
    pca = decomposition.PCA(n_components=3)
    pca.fit(normalised_features)
    X = pca.transform(normalised_features)

    # only needed for non int labels (I think)
    # y = np.choose(label, [1, 2, 0]).astype(np.float)

    pca_plot = ax1.scatter(X[:, 0], X[:, 1], X[:, 2], c=label, cmap=plt.cm.spectral)
    fig.colorbar(pca_plot, ax=ax1)
    # ticks
    ax1.w_xaxis.set_ticklabels([])
    ax1.w_yaxis.set_ticklabels([])
    ax1.w_zaxis.set_ticklabels([])
    
    # view perspective
    ax1.elev = 48
    ax1.azim = 134
    
    # reset angle std value
    normalised_features['angle std'] = normalised_features_backup['angle std']

    
#     #binary labels
#     labelReshape = label
#     labelReshape[labelReshape == 0] = 0
#     labelReshape[labelReshape == 1] = 1
#     labelReshape[labelReshape == 2] = 1 
#     labelReshape[labelReshape == 3] = 1
#     labelReshape[labelReshape == 4] = 1

    # STD PLOT #
    ax2 = fig.add_subplot(212)
    std_plot = ax2.scatter(std.index, std['angle std'], c=label, cmap=plt.cm.spectral)
    ax2.set_xlim(std.index[0], std.index[-1])
    fig.colorbar(std_plot, ax=ax2)
    
    return label, std 

    
widgets.interact(plots, weight= widgets.IntSlider(min=1,max=5,step=1,value=1))

In [ ]:
# plot labeled std 
plt.scatter(std.index, std['angle std'], c=label)
plt.xlim(std.index[0], std.index[-1])

### standardisation plot is better? TODO couldnt achieve 0 mean

In [ ]:
#standardise the features

scaled = preprocessing.scale(features)

### why can't I zero mean it?
# scaled_features.mean(axis=0)

scaled_features = pd.DataFrame(scaled)

# rename index and columns
scaled_features.columns = features.columns
scaled_features.index = features.index

scaled_features_backup = scaled_features.copy(deep=True)

In [ ]:
def plots_2(weight):
    

    scaled_features['angle std'] = scaled_features['angle std'] * weight
    label = KMeans(n_clusters=4, random_state=10).fit_predict(scaled_features)

    # PCA PLOT #
    fig = plt.figure(figsize=(10, 10))
    ax1 = fig.add_subplot(211, projection='3d')
    
    pca = decomposition.PCA(n_components=3)
    pca.fit(normalised_features)
    X = pca.transform(scaled_features)

    # only needed for non int labels (I think)
    # y = np.choose(label, [1, 2, 0]).astype(np.float)

    pca_plot = ax1.scatter(X[:, 0], X[:, 1], X[:, 2], c=label, cmap=plt.cm.spectral)
    fig.colorbar(pca_plot, ax=ax1)
    # ticks
    ax1.w_xaxis.set_ticklabels([])
    ax1.w_yaxis.set_ticklabels([])
    ax1.w_zaxis.set_ticklabels([])
    
    # view perspective
    ax1.elev = 48
    ax1.azim = 134
    
    # reset angle std value
    scaled_features['angle std'] = scaled_features_backup['angle std']

    
#     #binary labels
#     labelReshape = label
#     labelReshape[labelReshape == 0] = 0
#     labelReshape[labelReshape == 1] = 1
#     labelReshape[labelReshape == 2] = 1 
#     labelReshape[labelReshape == 3] = 1
#     labelReshape[labelReshape == 4] = 1

    # STD PLOT #
    ax2 = fig.add_subplot(212)
    std_plot = ax2.scatter(std.index, std['angle std'], c=label, cmap=plt.cm.spectral)
    ax2.set_xlim(std.index[0], std.index[-1])
    fig.colorbar(std_plot, ax=ax2)
    
    return label, std 

    
widgets.interact(plots_2, weight= widgets.IntSlider(min=1,max=5,step=1,value=1))

# Test whether 0-1 normalisation or standardization is better

In [ ]:
### count the number of labels with std below 10

# add it into a column into a dataframe